In [ ]:
import pandas as pd
import numpy as np
import csv
import nltk
from sklearn import svm
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from collections import defaultdict
from sklearn import model_selection, svm
from sklearn.ensemble import RandomForestClassifier
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

# data = pd.read_csv('train-balanced-sarcasm.csv', on_bad_lines="skip", engine="python", nrows = 100000)

# Read the labels and the data
labels = pd.read_csv('train-balanced-sarcasm.csv', dtype=int, usecols= ['label'], nrows = 100000)  
sarcasm_data = pd.read_csv('train-balanced-sarcasm.csv', dtype= str, usecols = ['comment'], nrows = 100000)

sarcasm_data = pd.DataFrame(sarcasm_data)
sarcasm_data['comment'].dropna(inplace=True)
sarcasm_data['comment'] = sarcasm_data['comment'].astype(str)

# make all comments lowercase
sarcasm_data['comment'] = sarcasm_data['comment'].str.lower()

# remove punctuations
def remove_punctuations(text):
    for char in string.punctuation:
        text = text.replace(char, '')
    return text
sarcasm_data['comment'] = sarcasm_data['comment'].apply(remove_punctuations)
# tokenization
sarcasm_data['comment'] = sarcasm_data['comment'].apply(word_tokenize)

# tag adjectives, verbs and adverbs for lemmatization
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ 
tag_map['V'] = wn.VERB 
tag_map['R'] = wn.ADV 

for i in range(sarcasm_data.shape[0]):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(sarcasm_data['comment'].iloc[i]):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    sarcasm_data.loc[i,'comment'] = str(Final_words)
    
print("Finished preprocessing")
print(sarcasm_data)


In [ ]:
# Split the data into validation, test, training
Train_X, Data_X, Train_Y, Data_Y = model_selection.train_test_split(sarcasm_data['comment'], labels['label'], test_size=0.3)
Test_X, Val_X, Test_Y, Val_Y = model_selection.train_test_split(Data_X,Data_Y,test_size=1/3)
# Encode the labels 
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)
# Vectorize the text data
Tfidf_vect = TfidfVectorizer(max_features=10000)
Tfidf_vect.fit(sarcasm_data['comment'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)
print(Tfidf_vect.vocabulary_)

In [ ]:
# SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Find the accuracy score
print("SVM Accuracy Score -> ", accuracy_score(predictions_SVM, Test_Y)*100)
print("Confusion Matrix for SVM:")
print(confusion_matrix(Test_Y, predictions_SVM, labels=[0,1]))

In [ ]:
# Random Forest
RFC = RandomForestClassifier()
# fit the training dataset on the classifier
RFC.fit(Train_X_Tfidf, Train_Y)
# predict the labels on validation set
predictions_RFC = RFC.predict(Test_X_Tfidf)
print(predictions_RFC)
# Calc. accuracy
print("RFC Accuracy Score ->", accuracy_score(predictions_RFC, Test_Y)*100)
print("Confusion Matrix for RFC:")
print(confusion_matrix(Test_Y, predictions_RFC, labels=[0,1]))

In [ ]:
#Naive Bayes 
NAIVE_BAYES = naive_bayes.MultinomialNB()
# fit the training dataset on the classifier
NAIVE_BAYES.fit(Train_X_Tfidf, Train_Y)
# predict the labels on validation set
predictions_NAIVE_BAYES = NAIVE_BAYES.predict(Test_X_Tfidf)
# Calc. accuracy
print("NAIVE BAYES Accuracy Score ->", accuracy_score(predictions_NAIVE_BAYES, Test_Y)*100)
print("Confusion Matrix for RFC:")
print(confusion_matrix(Test_Y, predictions_NAIVE_BAYES, labels=[0,1]))